In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GATConv
import time
from cora_loader import CitationNetwork,CocitationNetwork,ConfigurationModelCitationNetwork
import copy
import numpy as np

In [1]:
class MonoGAT(torch.nn.Module):
    def __init__(self,dataset,channels,heads=1):
        super(MonoGAT,self).__init__()
        channels = [dataset.num_node_features] + channels + [dataset.num_classes]
        self.conv = []
        for i in range(1,len(channels)):
            if i == 1:
                conv = GATConv(channels[i-1],channels[i],heads=heads)
            elif i == len(channels)-1:
                if dataset.name=='PubMed':
                    conv = GATConv(channels[i-1]*heads,channels[i],heads=heads,concat=False)
                else:
                    conv = GATConv(channels[i-1]*heads,channels[i])
            else:
                conv = GATConv(channels[i-1]*heads,channels[i],heads=heads)
            self.add_module(str(i),conv)
            self.conv.append(conv)
        
        
    def forward(self, data): 
        x, edge_index = data.x, data.edge_index 
        
        for conv in self.conv[:-1]:
            x = conv(x,edge_index)
            x = F.elu(x)
            x = F.dropout(x,p=0.6,training=self.training) # YOU MUST UNDERSTAND DROPOUT
        
        # Last layer
        x = self.conv[-1](x,edge_index)
        x = F.softmax(x,dim=1)
        
        return x
    

class BiGAT(torch.nn.Module):
    def __init__(self,dataset,channels,heads=1):
        super(BiGAT,self).__init__()
        self.conv_st = []
        self.conv_ts = []
        channels_output = [dataset.num_node_features] + [c*2*heads for c in channels]
        channels = [dataset.num_node_features] + channels
        for i in range(len(channels)-1):
            conv_st = GATConv(channels_output[i], channels[i+1],heads=heads)
            self.add_module('conv_st'+str(i),conv_st)
            self.conv_st.append(conv_st)
            
            conv_ts = GATConv(channels_output[i], channels[i+1],heads=heads)
            self.add_module('conv_ts'+str(i),conv_ts)
            self.conv_ts.append(conv_ts)
        
        if dataset.name=='PubMed':
            self.last = GATConv(channels_output[-1], dataset.num_classes,heads=heads,concat=False)
        else:
            self.last = GATConv(channels_output[-1], dataset.num_classes)
        
    def forward(self, data): 
        x, edge_index = data.x, data.edge_index
        st_edges = data.edge_index.t()[1-data.is_reversed].t()
        ts_edges = data.edge_index.t()[data.is_reversed].t()
#         print(ts_edges.shape)
        for i in range(len(self.conv_st)):
            x1 = F.elu(self.conv_st[i](x,st_edges))
            x2 = F.elu(self.conv_ts[i](x,ts_edges))
            x = torch.cat((x1,x2),dim=1)
            x = F.dropout(x,p=0.6,training=self.training)
        
        # last layer
        x = self.last(x,edge_index)
        x = F.softmax(x,dim=1) 
        
        return x
    
class TriGAT(torch.nn.Module):
    def __init__(self,dataset,channels,heads=1):
        super(TriGAT,self).__init__()
        self.conv_st = []
        self.conv_ts = []
        self.conv = []
        channels_output = [dataset.num_node_features] + [c*3*heads for c in channels]
        channels = [dataset.num_node_features] + channels
        for i in range(len(channels)-1):
            conv_st = GATConv(channels_output[i], channels[i+1],heads=heads)
            self.add_module('conv_st'+str(i),conv_st)
            self.conv_st.append(conv_st)
            
            conv_ts = GATConv(channels_output[i], channels[i+1],heads=heads)
            self.add_module('conv_ts'+str(i),conv_ts)
            self.conv_ts.append(conv_ts)
            
            conv = GATConv(channels_output[i],channels[i+1],heads=heads)
            self.add_module('conv'+str(i),conv)
            self.conv.append(conv)
        
        if dataset.name=='PubMed':
            self.last = GATConv(channels_output[-1], dataset.num_classes,heads=heads,concat=False)
        else:
            self.last = GATConv(channels_output[-1], dataset.num_classes)
        
    def forward(self, data): 
        x, edge_index = data.x, data.edge_index
        st_edges = data.edge_index.t()[1-data.is_reversed].t()
        ts_edges = data.edge_index.t()[data.is_reversed].t()
#         print(ts_edges.shape)
        for i in range(len(self.conv_st)):
            x1 = F.elu(self.conv_st[i](x,st_edges))
            x2 = F.elu(self.conv_ts[i](x,ts_edges))
            x3 = F.elu(self.conv[i](x,edge_index))
            x = torch.cat((x1,x2,x3),dim=1)
            x = F.dropout(x,training=self.training)
        
        # last layer
        x = self.last(x,edge_index)
        x = F.softmax(x,dim=1) 
        
        return x

NameError: name 'torch' is not defined

In [7]:

def run_and_eval_model(dataset,channels,architecture=MonoGAT,heads=1,epochs=200):
    # training process (without batches/transforms)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = architecture(dataset,channels,heads).to(device)
#     print(model)
    data = dataset[0].to(device)
    reg = 0.0001 if dataset.name=='PubMed' else 0.01
    lr = 0.1 if dataset.name=='PubMed' else 0.01
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=reg)
    model.train() # to enter training phase
    maxacc = 0
    chosen = None
    for epoch in range(epochs):
        optimizer.zero_grad() # saw this a lot in the beginning, maybe resetting gradients (not to accumulate)
        out = model(data) # this calls the forward method apparently
        loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask]) # nice indexing, easy and short
        loss.backward() # magic: real back propagation step, takes care of the gradients and stuff
        optimizer.step() # maybe updates the params to be optimized
        
        model.eval()
        _,pred = model(data).max(dim=1) # take prediction out of softmax
        correct = float(pred[data.val_mask].eq(data.y[data.val_mask]).sum().item())
        acc = correct / data.val_mask.sum().item()
        if acc > maxacc:
            maxacc = acc
            chosen=copy.deepcopy(model)
        model.train()
    chosen.eval() # enter eval phase
    _,pred = chosen(data).max(dim=1) # take prediction out of softmax
    correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    return acc

def eval_multiple(dataset,channels,architecture=MonoGAT,heads=1,runs=100,epochs=50):
    start = time.time()
    accs = []
    for i in range(runs):
        accs.append(run_and_eval_model(dataset,channels,architecture,heads=heads,epochs=epochs))
    elapsed_time = time.time() - start
    print('Elaplsed {}'.format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    return sum(accs)/len(accs),np.std(np.array(accs)),accs

In [2]:
cora = CitationNetwork('/tmp/cora','cora',directed=False)

In [3]:
cora = CitationNetwork('/tmp/cora2','cora',directed=False)

In [19]:
cora_full = CitationNetwork('/tmp/cora_full2','cora_full',directed=False)

In [20]:
cora_full.name

'cora_full'

In [15]:
cora[0].is_reversed.long().is_cuda

False

In [5]:
citeseer = CitationNetwork('/tmp/citeseer','citeseer',directed=False)

In [4]:
PubMed = CitationNetwork('/tmp/PubMed','PubMed',directed=False)

In [26]:
coraDirected = CitationNetwork('/tmp/coraDirected','cora',directed=True)
citeseerDirected = CitationNetwork('/tmp/citeseerDirected','citeseer',directed=True)
PubMedDirected = CitationNetwork('/tmp/PubMedDirected','PubMed',directed=True)

In [27]:
coraReversed = CitationNetwork('/tmp/coraReversed','cora',directed=True,reverse=True)
citeseerReversed = CitationNetwork('/tmp/citeseerReversed','citeseer',directed=True,reverse=True)
PubMedReversed = CitationNetwork('/tmp/PubMedReversed','PubMed',directed=True,reverse=True)

In [24]:
coraConfDirected = ConfigurationModelCitationNetwork('/tmp/coraConfDirected','cora',directed=True)
citeseerConfDirected = ConfigurationModelCitationNetwork('/tmp/citeseerConfDirected','citeseer',directed=True)
PubMedConfDirected = ConfigurationModelCitationNetwork('/tmp/PubMedConfDirected','PubMed',directed=True)

In [17]:
coraConfReversed = ConfigurationModelCitationNetwork('/tmp/coraConfReversed','cora',directed=True,reverse=True)
citeseerConfReversed = ConfigurationModelCitationNetwork('/tmp/citeseerConfReversed','citeseer',directed=True,reverse=True)
PubMedConfReversed = ConfigurationModelCitationNetwork('/tmp/PubMedConfReversed','PubMed',directed=True,reverse=True)

In [40]:
coraConf = ConfigurationModelCitationNetwork('/tmp/coraConf','cora',directed=False)
citeseerConf = ConfigurationModelCitationNetwork('/tmp/citeseerConf','citeseer',directed=False)
PubMedConf = ConfigurationModelCitationNetwork('/tmp/PubMedConf','PubMed',directed=False)

## Directed

In [34]:
acc,stdAcc,accs = eval_multiple(coraDirected,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('coraDirected accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerDirected,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('citeseerDirected accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(PubMedDirected,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('PubMedDirected accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:08:21
coraDirected accuracy: 0.7336+-0.0131
Elaplsed 00:03:36
citeseerDirected accuracy: 0.7031+-0.0063
Elaplsed 00:05:25
PubMedDirected accuracy: 0.6752+-0.0138


## Reversed

In [35]:
acc,stdAcc,accs = eval_multiple(coraReversed,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('coraReversed accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerReversed,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('citeseerReversed accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(PubMedReversed,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('PubMedReversed accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:05:01
coraReversed accuracy: 0.6673+-0.0133
Elaplsed 00:04:58
citeseerReversed accuracy: 0.6532+-0.0077
Elaplsed 00:04:16
PubMedReversed accuracy: 0.6834+-0.0059


### Directed Conf Model

In [36]:
acc,stdAcc,accs = eval_multiple(coraConfDirected,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('coraConfDirected accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerConfDirected,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)
print('citeseerConfDirected accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))


Elaplsed 00:03:00
coraConfDirected accuracy: 0.1971+-0.0197
Elaplsed 00:04:16
citeseerConfDirected accuracy: 0.2840+-0.0146


KeyboardInterrupt: 

In [41]:

acc,stdAcc,accs = eval_multiple(PubMedConfDirected,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('PubMedConfDirected accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:01:58
PubMedConfDirected accuracy: 0.4283+-0.0085


### Reversed Conf Model

In [42]:
acc,stdAcc,accs = eval_multiple(coraConfReversed,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('coraConfReversed accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerConfReversed,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('citeseerConfReversed accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(PubMedConfReversed,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('PubMedConfReversed accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:01:32
coraConfReversed accuracy: 0.3052+-0.0148
Elaplsed 00:01:29
citeseerConfReversed accuracy: 0.3167+-0.0166
Elaplsed 00:01:54
PubMedConfReversed accuracy: 0.5844+-0.0145


### Undirected Conf Model

In [43]:
acc,stdAcc,accs = eval_multiple(coraConf,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('coraConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerConf,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('citeseerConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(PubMedConf,[32],architecture=MonoGAT,heads=2,runs=50,epochs=200)
print('PubMedConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:01:31
coraConf accuracy: 0.2274+-0.0499
Elaplsed 00:01:31
citeseerConf accuracy: 0.3597+-0.0200
Elaplsed 00:02:21
PubMedConf accuracy: 0.4923+-0.0160


In [44]:
acc,stdAcc,accs = eval_multiple(coraConf,[32],architecture=BiGAT,heads=2,runs=50,epochs=200)
print('coraConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerConf,[32],architecture=BiGAT,heads=2,runs=50,epochs=200)
print('citeseerConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(PubMedConf,[32],architecture=BiGAT,heads=2,runs=50,epochs=200)
print('PubMedConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:02:16
coraConf accuracy: 0.2736+-0.0262
Elaplsed 00:02:19
citeseerConf accuracy: 0.3376+-0.0190
Elaplsed 00:03:14
PubMedConf accuracy: 0.5490+-0.0315


In [45]:
acc,stdAcc,accs = eval_multiple(coraConf,[32],architecture=TriGAT,heads=2,runs=50,epochs=200)
print('coraConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(citeseerConf,[32],architecture=TriGAT,heads=2,runs=50,epochs=200)
print('citeseerConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))
acc,stdAcc,accs = eval_multiple(PubMedConf,[32],architecture=TriGAT,heads=2,runs=50,epochs=200)
print('PubMedConf accuracy: {:.4f}+-{:.4f}'.format(acc,stdAcc))

Elaplsed 00:02:55
coraConf accuracy: 0.2751+-0.0306
Elaplsed 00:02:59
citeseerConf accuracy: 0.3504+-0.0209
Elaplsed 00:05:07
PubMedConf accuracy: 0.5440+-0.0307


### Architecture reported by the authors

#### Cora

In [11]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[16],architecture=BiGAT,heads=2,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[10],architecture=TriGAT,heads=2,runs=100,epochs=200)[0]))

Elaplsed 00:03:50
cora accuracy: 0.8091
Elaplsed 00:05:15
cora accuracy: 0.8104
Elaplsed 00:07:01
cora accuracy: 0.8133


AFTER param_search

In [7]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[3],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:05:13
cora accuracy: 0.8097
Elaplsed 00:09:51
cora accuracy: 0.8077
Elaplsed 00:15:26
cora accuracy: 0.8148


Older: before param_search

In [42]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[3],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:03:51
cora accuracy: 0.7968
Elaplsed 00:06:13
cora accuracy: 0.7911
Elaplsed 00:08:04
cora accuracy: 0.7950


In [47]:

print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[2],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:05:56
cora accuracy: 0.7949


#### CiteSeer

In [12]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[16],architecture=BiGAT,heads=2,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[10],architecture=TriGAT,heads=2,runs=100,epochs=200)[0]))

Elaplsed 00:03:30
citeseer accuracy: 0.7364
Elaplsed 00:04:35
citeseer accuracy: 0.7318
Elaplsed 00:05:59
citeseer accuracy: 0.7333


AFTER param_search

In [8]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[3],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:14:29
citeseer accuracy: 0.7377
Elaplsed 00:08:29
citeseer accuracy: 0.7334
Elaplsed 00:08:55
citeseer accuracy: 0.7337


Older: before param_search

In [44]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[3],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:04:44
citeseer accuracy: 0.7212
Elaplsed 00:06:40
citeseer accuracy: 0.7270
Elaplsed 00:09:10
citeseer accuracy: 0.7258


In [46]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[2],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:09:26
citeseer accuracy: 0.7280


#### PubMed

After param search for PubMed

In [8]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[2],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:07:20
PubMed accuracy: 0.7762
Elaplsed 00:07:43
PubMed accuracy: 0.7545
Elaplsed 00:09:35
PubMed accuracy: 0.7626


In [13]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[32],architecture=MonoGAT,heads=2,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[16],architecture=BiGAT,heads=2,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[10],architecture=TriGAT,heads=2,runs=100,epochs=200)[0]))

Elaplsed 00:05:37
PubMed accuracy: 0.7686
Elaplsed 00:06:17
PubMed accuracy: 0.7502
Elaplsed 00:07:37
PubMed accuracy: 0.7573


AFTER param_search

In [10]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[2],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:09:41
PubMed accuracy: 0.7718
Elaplsed 00:11:09
PubMed accuracy: 0.7461
Elaplsed 00:11:15
PubMed accuracy: 0.7537


Older: before param_search

In [56]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[8],architecture=MonoGAT,heads=8,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[4],architecture=BiGAT,heads=8,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[2],architecture=TriGAT,heads=8,runs=100,epochs=200)[0]))

Elaplsed 00:12:29
PubMed accuracy: 0.7708
Elaplsed 00:07:31
PubMed accuracy: 0.7446
Elaplsed 00:09:15
PubMed accuracy: 0.7542


### Single-head architecture with same representation size

#### Cora

In [43]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[64],architecture=MonoGAT,heads=1,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[32],architecture=BiGAT,heads=1,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[20],architecture=TriGAT,heads=1,runs=100,epochs=200)[0]))

Elaplsed 00:04:33
cora accuracy: 0.7838
Elaplsed 00:06:29
cora accuracy: 0.7912
Elaplsed 00:08:14
cora accuracy: 0.7920


#### CiteSeer

In [45]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[64],architecture=MonoGAT,heads=1,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[32],architecture=BiGAT,heads=1,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[20],architecture=TriGAT,heads=1,runs=100,epochs=200)[0]))

Elaplsed 00:03:19
citeseer accuracy: 0.7170
Elaplsed 00:04:36
citeseer accuracy: 0.7232
Elaplsed 00:05:51
citeseer accuracy: 0.7245


#### PubMed

In [57]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[64],architecture=MonoGAT,heads=1,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[32],architecture=BiGAT,heads=1,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMed,[20],architecture=TriGAT,heads=1,runs=100,epochs=200)[0]))

Elaplsed 00:04:47
PubMed accuracy: 0.7601
Elaplsed 00:05:23
PubMed accuracy: 0.7488
Elaplsed 00:06:40
PubMed accuracy: 0.7533
